In [1]:
import torch


def load_data():
    import PIL.Image
    import numpy as np
    import os

    xs = []
    ys = []

    #遍历文件夹下的所有文件
    for filename in os.listdir('data/cifar10'):

        #只要图片,过滤一些无关的文件
        if not filename.endswith('.jpg'):
            continue

        #读取图片信息
        x = PIL.Image.open('data/cifar10/%s' % filename)

        #转矩阵,数值压缩到0-1之间
        x = torch.FloatTensor(np.array(x)) / 255

        #变形,把通道放前面
        #[32, 32, 3] -> [3, 32, 32]
        x = x.permute(2, 0, 1)

        #y来自文件名的第一个字符
        y = int(filename[0])

        xs.append(x)
        ys.append(y)

    return xs, ys


xs, ys = load_data()

len(xs), len(ys), xs[0].shape, ys[0]

(60000, 60000, torch.Size([3, 32, 32]), 9)

In [2]:
#定义数据集
class Dataset(torch.utils.data.Dataset):

    def __len__(self):
        return len(xs)

    def __getitem__(self, i):
        return xs[i], ys[i]


dataset = Dataset()

x, y = dataset[0]

len(dataset), x.shape, y

(60000, torch.Size([3, 32, 32]), 9)

In [3]:
#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=8,
                                     shuffle=True,
                                     drop_last=True)

x, y = next(iter(loader))

len(loader), x.shape, y

(7500, torch.Size([8, 3, 32, 32]), tensor([5, 1, 6, 6, 4, 7, 5, 1]))

In [4]:
#cnn神经网络
class Model(torch.nn.Module):

    def __init__(self):
        super().__init__()

        #520的卷积层
        self.cnn1 = torch.nn.Conv2d(in_channels=3,
                                    out_channels=16,
                                    kernel_size=5,
                                    stride=2,
                                    padding=0)

        #311的卷积层
        self.cnn2 = torch.nn.Conv2d(in_channels=16,
                                    out_channels=32,
                                    kernel_size=3,
                                    stride=1,
                                    padding=1)

        #710的卷积层
        self.cnn3 = torch.nn.Conv2d(in_channels=32,
                                    out_channels=128,
                                    kernel_size=7,
                                    stride=1,
                                    padding=0)

        #池化层
        self.pool = torch.nn.MaxPool2d(kernel_size=2, stride=2)

        #激活函数
        self.relu = torch.nn.ReLU()

        #线性输出层
        self.fc = torch.nn.Linear(in_features=128, out_features=10)

    def forward(self, x):

        #第一次卷积,形状变化可以推演
        #[8, 3, 32, 32] -> [8, 16, 14, 14]
        x = self.cnn1(x)
        x = self.relu(x)

        #第二次卷积,因为是311的卷积,所以尺寸不变
        #[8, 16, 14, 14] -> [8, 32, 14, 14]
        x = self.cnn2(x)
        x = self.relu(x)

        #池化,尺寸减半
        #[8, 32, 14, 14] -> [8, 32, 7, 7]
        x = self.pool(x)

        #第三次卷积,因为核心是7,所以只有一步计算
        #[8, 32, 7, 7] -> [8, 128, 1, 1]
        x = self.cnn3(x)
        x = self.relu(x)

        #展平,便于线性计算,也相当于把图像变成向量
        #[8, 128, 1, 1] -> [8, 128]
        x = x.flatten(start_dim=1)

        #线性计算输出
        #[8, 128] -> [8, 10]
        return self.fc(x)


model = Model()

model(torch.randn(8, 3, 32, 32)).shape

torch.Size([8, 10])

In [5]:
#训练
def train():
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    loss_fun = torch.nn.CrossEntropyLoss()
    model.train()

    for epoch in range(5):
        for i, (x, y) in enumerate(loader):
            out = model(x)
            loss = loss_fun(out, y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            if i % 2000 == 0:
                acc = (out.argmax(dim=1) == y).sum().item() / len(y)
                print(epoch, i, loss.item(), acc)

    torch.save(model, 'model/5.model')


train()

0 0 2.276381254196167 0.0
0 2000 1.7430951595306396 0.375
0 4000 1.50967276096344 0.5
0 6000 1.379901647567749 0.625
1 0 0.7615889310836792 0.75
1 2000 0.7796550393104553 0.75
1 4000 0.9899455308914185 0.625
1 6000 1.0108083486557007 0.5
2 0 1.21652352809906 0.25
2 2000 0.9723398685455322 0.625
2 4000 0.6790870428085327 0.75
2 6000 1.51291823387146 0.5
3 0 0.6330904960632324 0.75
3 2000 1.2843165397644043 0.75
3 4000 1.1586779356002808 0.625
3 6000 0.5945857763290405 0.75
4 0 1.0815905332565308 0.625
4 2000 0.8424673080444336 0.625
4 4000 0.6966819763183594 0.75
4 6000 0.9041645526885986 0.5


In [6]:
#测试
@torch.no_grad()
def test():
    model = torch.load('model/5.model')
    model.eval()

    correct = 0
    total = 0
    for i in range(100):
        x, y = next(iter(loader))

        out = model(x).argmax(dim=1)

        correct += (out == y).sum().item()
        total += len(y)

    print(correct / total)


test()

0.785
